In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [6]:
import os

from unsloth import FastLanguageModel
import torch
import random # For A/B assignment

import pandas as pd
from datasets import Dataset
import re
import numpy as np
import random # Ensure random is imported

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [55]:


TARGET_NOTES_LENGTH = 128


max_seq_length = 1536
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-1.5B-Instruct", # Or "Qwen/Qwen1.5-7B-Chat", "mistralai/Mistral-7B-Instruct-v0.2" etc.
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=False,
    max_lora_rank=lora_rank,
)


model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=lora_rank,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)




==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.1. vLLM: 0.8.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

In [56]:

SYSTEM_PROMPT = f"""
REQUEST: You are an expert academic editor comparing two potential titles for a research abstract.
You must judge which of the two titles, Sample A or Sample B, is Funnier, considering the provided abstract.

Respond with an exactly 128 tokens tag labeled <notes> that contains your notes, and then <judgement> which is just the letter that you are picking.

For example:

JUDGE: <notes>
Sample A is superior to Sample B... (example notes)
</notes>
<judgement>A</judgement>

Now, it is your turn."""


In [42]:

df = pd.read_csv('/content/humor_classification_results.csv')
df.dropna(subset=['abstract', 'original_title', 'created_title', 'humor_category'], inplace=True)


df['target_label'] = 'original_title'
raw_dataset = Dataset.from_pandas(df)
print(f"Loaded {len(raw_dataset)} examples from /content/humor_classification_results.csv")

Loaded 488 examples from /content/humor_classification_results.csv


In [59]:



def format_title_AB_prompt(example):
    original = example['original_title']
    created = example['created_title']
    target_label = example['target_label']
    assignment = {}
    target_choice = None

    if random.random() < 0.5:

        sample_a_content = original
        sample_b_content = created
        assignment['A'] = 'original_title'
        assignment['B'] = 'created_title'
        target_choice = 'B' if target_label == 'created_title' else 'A'
    else:
        sample_a_content = created
        sample_b_content = original
        assignment['A'] = 'created_title'
        assignment['B'] = 'original_title'
        target_choice = 'A' if target_label == 'created_title' else 'B'

    user_content = f"""Abstract:
{example['abstract']}

[Sample A]:
{sample_a_content}

[Sample B]:
{sample_b_content}

JUDGE:"""

    prompt_list = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
    ]

    example['prompt'] = prompt_list
    example['assignment'] = assignment
    example['target_choice'] = target_choice

    return example

dataset = raw_dataset.map(format_title_AB_prompt)



Map:   0%|          | 0/488 [00:00<?, ? examples/s]

In [60]:
print("\n--- Example Formatted Data ---")
print(tokenizer.apply_chat_template(dataset[4]['prompt'], tokenize=False, add_generation_prompt=True)) # Assumes tokenizer is defined
print(f"Assignment: {dataset[4]['assignment']}")
print(f"Target Label: {dataset[4]['target_label']}")
print(f"==> Target Choice for Rewards: {dataset[4]['target_choice']}")
print("-" * 30)


--- Example Formatted Data ---
<|im_start|>system

REQUEST: You are an expert academic editor comparing two potential titles for a research abstract.
You must judge which of the two titles, Sample A or Sample B, is Funnier, considering the provided abstract.

Respond with an exactly 128 tokens tag labeled <notes> that contains your notes, and then <judgement> which is just the letter that you are picking.

For example:

JUDGE: <notes>
Sample A is superior to Sample B... (example notes)
</notes>
<judgement>A</judgement>

Now, it is your turn.<|im_end|>
<|im_start|>user
Abstract:
Corpus query systems exist to address the multifarious information needs of any person interested in the content of annotated corpora. In this role they play an important part in making those resources usable for a wider audience. Over the past decades, several such query systems and languages have emerged, varying greatly in their expressiveness and technical details. This paper offers a broad overview of the 

In [62]:
def extract_tags(text: str) -> tuple[str | None, str | None]:
    notes_match = re.search(r"<notes>(.*?)</notes>", text, re.DOTALL | re.IGNORECASE)
    judgement_match = re.search(r"<judgement>\s*([AB])\s*</judgement>", text, re.IGNORECASE)

    notes = notes_match.group(1).strip() if notes_match else None
    judgement = judgement_match.group(1).strip().upper() if judgement_match else None
    return notes, judgement

print_counter = 0
PRINT_EVERY_N_STEPS = 5

def judgement_reward_func(prompts, completions, target_choice, **kwargs) -> list[float]:
    """
    Calculates reward based on judgement correctness.
    Also prints prompt and first completion periodically during training.
    """
    global print_counter


    if not target_choice or not target_choice[0] in ['A', 'B']:
        print(f"Warning: Invalid target_choice in judgement_reward_func: {target_choice}")
        return [0.0] * len(completions)
    target = target_choice[0]

    rewards = []
    log_this_step = (print_counter % PRINT_EVERY_N_STEPS == 0)


    if log_this_step and prompts:
        try:
            prompt_text = tokenizer.apply_chat_template(
                prompts[0],
                tokenize=False,
                add_generation_prompt=False
            )
            print("\n" + "="*20 + f" Training Step Log (Print Step {print_counter}) " + "="*20)
            print("\n--- Prompt ---")
            print(prompt_text)

            if completions and completions[0]:
                 first_completion_text = completions[0][0]["content"]
                 print("\n--- Generated Response [0] ---")
                 print(first_completion_text)
            else:
                 print("\n--- Generated Response [0]: (Not available) ---")
            print("="*70)

        except Exception as e:
            print(f"\nError during logging in reward function: {e}")

    for i, completion in enumerate(completions):
        current_reward = 0.0
        try:
            text = completion[0]["content"]
            _, judgement = extract_tags(text)

            if judgement == target:
                current_reward = 1.0
            else:
                current_reward = -0.1

        except Exception as e:
            print(f"Warning: Error calculating reward for completion {i}: {e}")
            current_reward = 0.0

        rewards.append(current_reward)

    print_counter += 1
    return rewards


def format_bonus_reward_func(completions, target_choice, **kwargs) -> list[float]:
    target = target_choice[0]
    rewards = []
    format_pattern = r"^\s*<notes>.*?</notes>\s*<judgement>\s*[AB]\s*</judgement>\s*$"

    for i, completion in enumerate(completions):
        text = completion[0]["content"] if isinstance(completion, list) else completion.get("content", "")
        notes, judgement = extract_tags(text)
        matches_format = bool(re.match(format_pattern, text, re.DOTALL | re.IGNORECASE))

        reward = 0.0
        if matches_format:
            reward = 0.2

        rewards.append(reward)

    return rewards


def length_bonus_reward_func_simplified(completions, target_choice, **kwargs) -> list[float]:
    """
    Provides a reward bonus ONLY IF:
    1. The generated judgement matches the target_choice.
    2. The <notes> content exists.
    3. The CHARACTER length of the <notes> content is within a tolerance
       range of TARGET_NOTES_LENGTH (defined in characters).

    The reward decreases linearly from 0.2 (at target length) to 0 (at tolerance edge).
    """

    if not target_choice or not target_choice[0] in ['A', 'B']:
        print(f"Warning: Invalid target_choice received: {target_choice}")
        return [0.0] * len(completions)
    target_judgement = target_choice[0]

    rewards = []
    target_len_chars = TARGET_NOTES_LENGTH
    tolerance_ratio = 0.3
    max_deviation = target_len_chars * tolerance_ratio
    min_len_chars = target_len_chars - max_deviation
    max_len_chars = target_len_chars + max_deviation
    max_reward = 0.2

    for completion in completions:
        current_reward = 0.0
        try:
            text = completion[0]["content"]
            notes, generated_judgement = extract_tags(text)

            if generated_judgement == target_judgement:
                if notes is not None:
                    notes_char_length = len(notes)

                    if min_len_chars <= notes_char_length <= max_len_chars:
                        deviation = abs(notes_char_length - target_len_chars)


                        if max_deviation > 0:
                            current_reward = max_reward * (1 - (deviation / max_deviation))
                        else:
                            current_reward = max_reward if deviation == 0 else 0.0

        except (IndexError, TypeError, KeyError, AttributeError) as e:
            print(f"Warning: Error processing completion: {e}. Completion: {completion}")
            current_reward = 0.0

        rewards.append(current_reward)



    return rewards


In [68]:
from trl import GRPOConfig, GRPOTrainer
from transformers import TrainingArguments
import torch
import datasets

training_args = GRPOConfig(
    output_dir="outputs_title_eval_kalomaze_corrected",
    num_train_epochs=1,
    max_steps=300,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-6,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    logging_steps=2,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    report_to="wandb",
    remove_unused_columns=False,
    max_prompt_length= 1010,
    num_generations=4,
    max_completion_length=173,
    beta=0.04,
    max_grad_norm=0.2,
    eval_steps=eval_steps,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [69]:

trainer = GRPOTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    reward_funcs=[
        judgement_reward_func,
        format_bonus_reward_func,
        length_bonus_reward_func_simplified,
    ],
)

print("Starting GRPO training (Corrected Params)...")

try:
    trainer.train()
    print("Training finished.")
except Exception as e:
    print(f"An error occurred during training: {e}")

Starting GRPO training (Corrected Params)...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 439 | Num Epochs = 3 | Total steps = 300
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 36,929,536/1,580,643,840 (2.34% trained)



==================== Training Step Log (Print Step 40) ====================

--- Prompt ---
<|im_start|>system

REQUEST: You are an expert academic editor comparing two potential titles for a research abstract.
You must judge which of the two titles, Sample A or Sample B, is Funnier, considering the provided abstract.

Respond with an exactly 128 tokens tag labeled <notes> that contains your notes, and then <judgement> which is just the letter that you are picking.

For example:

JUDGE: <notes>
Sample A is superior to Sample B... (example notes)
</notes>
<judgement>A</judgement>

Now, it is your turn.<|im_end|>
<|im_start|>user
Abstract:
State-of-the-art methods for Word Sense Disambiguation (WSD) combine two different features: the power of pre-trained language models and a propagation method to extend the coverage of such models. This propagation is needed as current sense-annotated corpora lack coverage of many instances in the underlying sense inventory (usually WordNet). At the s

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / judgement_reward_func,rewards / format_bonus_reward_func,rewards / length_bonus_reward_func_simplified
2,0.000000,0.446875,0.585853,57.156250,0.000750,0.415625,0.031250,0.000000
4,0.000000,0.558105,0.520741,46.062500,0.001128,0.518750,0.037500,0.001855
6,0.000100,0.256250,0.367315,62.656250,0.001304,0.243750,0.012500,0.000000
8,0.000000,0.412500,0.533812,53.375000,0.000922,0.381250,0.031250,0.000000
10,0.000100,0.275000,0.451942,57.468750,0.001303,0.243750,0.031250,0.000000
12,0.000000,0.384375,0.551348,73.593750,0.001220,0.346875,0.037500,0.000000
14,0.000000,0.193750,0.450987,64.187500,0.000937,0.175000,0.018750,0.000000
16,0.000000,0.419629,0.462661,59.000000,0.000870,0.381250,0.037500,0.000879
18,0.000000,0.187500,0.374702,66.750000,0.000983,0.175000,0.012500,0.000000
20,0.000000,0.434375,0.540838,55.093750,0.000836,0.415625,0.018750,0.000000


Streaming output truncated to the last 5000 lines.
[Sample A]:
Learning Prepositional Paraphrases for Noun Compounds through LSTM-Based Sequence Modeling

[Sample B]:
Treat us like the sequences we are: Prepositional Paraphrasing of Noun Compounds using LSTM

JUDGE:<|im_end|>


--- Generated Response [0] ---
<notes>
Sample B is clever and plays with the metaphor of treating sequences like themselves. It seems to highlight the self-referential nature of the approach, which is intriguing and engaging. Sample A, while clear and straightforward, lacks this playfulness.
</notes>
<judgement>B</judgement>

==================== Training Step Log (Print Step 590) ====================

--- Prompt ---
<|im_start|>system

REQUEST: You are an expert academic editor comparing two potential titles for a research abstract.
You must judge which of the two titles, Sample A or Sample B, is Funnier, considering the provided abstract.

Respond with an exactly 128 tokens tag labeled <notes> that contains yo

In [70]:

trainer.save_model() # Save LoRA adapter
print(f"LoRA adapters saved to {training_args.output_dir}")

# Prepare a test example


LoRA adapters saved to outputs_title_eval_kalomaze_corrected


## Evaluation Code

In [73]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList
from tqdm.notebook import tqdm # For progress bar
import re
import pandas as pd # For displaying results later

# --- Make sure training is finished before running this ---
# trainer.train() call should be completed above this section.

print("\n--- Starting Manual Evaluation on Eval Dataset ---")

# --- Reuse Stopping Criteria Setup from Testing ---
# (Ensure this is defined/available in your notebook scope)
class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids_list):
        # Ensure stop_token_ids_list is a list of tensors
        self.stop_token_ids_list = stop_token_ids_list

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # Check if the *end* of the generated sequence matches any of the stop sequences
        for stop_ids in self.stop_token_ids_list:
            if input_ids.shape[1] >= stop_ids.shape[0]:
                if torch.eq(input_ids[0, -stop_ids.shape[0]:], stop_ids).all():
                    return True # Stop generation
        return False # Continue generation

# Tokenize the desired stop string '</judgement>'
stop_string = "</judgement>"
# Ensure tokenizer and model are on the same device
device = model.device # Get device from model
stop_token_ids = tokenizer.encode(stop_string, add_special_tokens=False, return_tensors='pt').to(device)

# Also include the standard EOS token ID tensor
eos_token_id_tensor = torch.tensor([[tokenizer.eos_token_id]], device=device)

# Create the stopping criteria list
# Pass a list containing tensors for each stop sequence
stop_criteria = StoppingCriteriaList([StopOnTokens([stop_token_ids[0], eos_token_id_tensor[0]])])

# --- Helper Function to Extract Tags (ensure this matches your testing code) ---
# --- Helper Function to Extract Tags (Corrected Regex) ---
def extract_tags(text):
    # Regex for <notes> seems fine if it captures until </notes>
    notes_match = re.search(r"<notes>(.*?)</notes>", text, re.DOTALL | re.IGNORECASE)

    # --- Corrected Regex for <judgement> ---
    # Make the closing </judgement> tag optional
    # It looks for <judgement>, optional whitespace, [AB], optional whitespace, then the optional </judgement>
    judgement_match = re.search(r"<judgement>\s*([AB])\s*(?:</judgement>)?", text, re.DOTALL | re.IGNORECASE)

    notes = notes_match.group(1).strip() if notes_match else None
    judgement = judgement_match.group(1).strip() if judgement_match else None # Still strip whitespace around A/B
    return notes, judgement

# --- Evaluation Loop ---
# ... (rest of your evaluation code remains the same)
# --- Evaluation Loop ---
results = []
correct_judgements = 0
format_matches = 0
total_evaluated = 0

# Regex for format checking (adjust if needed)
format_pattern = r"^\s*<notes>.*?</notes>\s*<judgement>\s*[AB]\s*</judgement>\s*$"

# Ensure model is in evaluation mode (disables dropout, etc.)
model.eval()

# Use torch.no_grad() to save memory and computation during inference
with torch.no_grad():
    # Loop through the evaluation dataset with a progress bar
    for item in tqdm(eval_dataset, desc="Evaluating"):
        total_evaluated += 1
        inference_prompt_chat = item['prompt'] # Get pre-formatted prompt
        inference_prompt_text = tokenizer.apply_chat_template(
            inference_prompt_chat,
            tokenize=False,
            add_generation_prompt=True # Add the prompt for generation
        )
        target_choice = item['target_choice'] # Ground truth ('A' or 'B')

        # Tokenize input
        inputs = tokenizer(
            inference_prompt_text,
            return_tensors="pt",
            truncation=True,
            max_length=1010 # Use same max_prompt_length as training
        ).to(device)

        # Generate response
        try:
            outputs = model.generate(
                **inputs,
                max_new_tokens=178, # Use same max_completion_length
                pad_token_id=tokenizer.eos_token_id,
                # eos_token_id=tokenizer.eos_token_id, # EOS handled by stopping criteria
                stopping_criteria=stop_criteria,
                do_sample=True, # Use same sampling params as testing for consistency
                temperature=0.6,
                top_p=0.9,
            )
            # Decode generated part
            generated_text = tokenizer.decode(outputs[0, inputs.input_ids.shape[1]:], skip_special_tokens=True)

            # Optional: Clean the stop string if present at the end
            if generated_text.endswith(stop_string):
                 generated_text = generated_text[:-len(stop_string)]

            # Analyze the generated text
            notes, judgement = extract_tags(generated_text)
            is_correct = False
            matches_format = bool(re.match(format_pattern, generated_text, re.DOTALL | re.IGNORECASE))

            if judgement and judgement == target_choice:
                correct_judgements += 1
                is_correct = True

            if matches_format:
                 format_matches += 1

            # Store results
            results.append({
                "prompt_text": inference_prompt_text,
                "generated_text": generated_text,
                "extracted_notes": notes,
                "extracted_judgement": judgement,
                "target_choice": target_choice,
                "is_correct": is_correct,
                "matches_format": matches_format
            })

        except Exception as e:
            print(f"\nError processing item {total_evaluated}: {e}")
            # Store minimal info for errored items
            results.append({
                "prompt_text": inference_prompt_text,
                "generated_text": f"ERROR: {e}",
                "extracted_notes": None,
                "extracted_judgement": None,
                "target_choice": target_choice,
                "is_correct": False,
                "matches_format": False
            })


# --- Calculate and Print Final Metrics ---
print("\n--- Evaluation Summary ---")
if total_evaluated > 0:
    accuracy = (correct_judgements / total_evaluated) * 100
    format_match_rate = (format_matches / total_evaluated) * 100
    print(f"Total Items Evaluated: {total_evaluated}")
    print(f"Correct Judgements: {correct_judgements}")
    print(f"Judgement Accuracy: {accuracy:.2f}%")
    print(f"Format Matches: {format_matches}")
    print(f"Format Match Rate: {format_match_rate:.2f}%")
else:
    print("No items were evaluated.")

# --- Optional: Display some results in a DataFrame ---
print("\n--- Sample Evaluation Results ---")
results_df = pd.DataFrame(results)
# Display head and maybe some incorrect examples
print(results_df.head())

incorrect_samples = results_df[results_df['is_correct'] == False].head()
if not incorrect_samples.empty:
    print("\n--- Sample Incorrect Judgements ---")
    for index, row in incorrect_samples.iterrows():
        print(f"\nItem {index}:")
        # print(f"Prompt:\n{row['prompt_text']}") # Can be very long
        print(f"Target: {row['target_choice']}")
        print(f"Generated Judgement: {row['extracted_judgement']}")
        print(f"Generated Text:\n{row['generated_text']}")
else:
     print("\nNo incorrect judgements found in the sample.")


--- Starting Manual Evaluation on Eval Dataset ---


Evaluating:   0%|          | 0/49 [00:00<?, ?it/s]


--- Evaluation Summary ---
Total Items Evaluated: 49
Correct Judgements: 30
Judgement Accuracy: 61.22%
Format Matches: 0
Format Match Rate: 0.00%

--- Sample Evaluation Results ---
                                         prompt_text  \
0  <|im_start|>system\n\nREQUEST: You are an expe...   
1  <|im_start|>system\n\nREQUEST: You are an expe...   
2  <|im_start|>system\n\nREQUEST: You are an expe...   
3  <|im_start|>system\n\nREQUEST: You are an expe...   
4  <|im_start|>system\n\nREQUEST: You are an expe...   

                                      generated_text  \
0  <notes>\nThe title "Reading Like HER: Human Re...   
1  <notes>\nThe title "Improving the Stability of...   
2  <notes>\nSample A captures the essence of pred...   
3  <notes>\nThe title "It's All in the Name: Miti...   
4  <notes>\nThe title "Humor in Word Embeddings: ...   

                                     extracted_notes extracted_judgement  \
0  The title "Reading Like HER: Human Reading Ins...                